Q1: You need to implement Naïve Bayes Classifier on sentiment classification dataset (IMDB
Dataset).
Calculate accuracy, precision, recall, f-1 score, micro-avg, macro-avg and 10-fold validation.

IMDB Movie Ratings Sentiment Analysis Dataset

Q2: You need to implement any three Classification algorithms from Sklearn on text classification
datasets, you can use any publicly available dataset.
Calculate accuracy, precision, recall, f-1 score, micro-avg, macro-avg and 10-fold validation.
*An implemented reference notebook and dataset attached below for syntax familiarity. 


In [24]:
from itertools import chain

import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords', quiet=True)
# nltk.download('punkt', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import numpy as np
import pandas as pd

import seaborn as sns

from sklearn import feature_extraction
from sklearn.metrics import confusion_matrix
from sklearn import model_selection as ms
from sklearn import naive_bayes
from sklearn import preprocessing
from sklearn.metrics import classification_report

import string

In [25]:
text_df = pd.read_csv('/content/IMDB Movie Ratings Sentiment Analysis Dataset.zip')
text_df.head(5)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [26]:
text_df.head(15)
text_df.tail(15)

,text,label
39985,Comedy? What's so funny about watching an ugly...,0
39986,Best club scenes that i have seen in a long ti...,1
39987,I have great memories of this movie...<br /><b...,1
39988,An underrated addition to the Graham Greene ci...,1
39989,Walter Matthau and George Burns just work so w...,1
39990,I really love anything done by Savage Steve Ho...,1
39991,Revenge is the theme of this Denzel Washington...,1
39992,Although I agree that it's a good but not grea...,1
39993,"""The Best Movie of the 90's"" ""The Welsh Trains...",0
39994,My girlfriend has the habit of going to Blockb...,0


In [27]:
categories = list(text_df['text'].unique())
categories

['I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.',
 "When I put this movie in my DVD player, and sat down with a coke and some chips, I had some expectations. I was hoping 

In [15]:
text_df.shape

(40000, 2)

In [29]:
cleaned_website_text = list(text_df['label'].unique())
len(cleaned_website_text)

2

In [28]:
text_df['text'].value_counts()

Hilarious, clean, light-hearted, and quote-worthy. What else can you ask for in a film? This is my all-time, number one favorite movie. Ever since I was a little girl, I've dreamed of owning a blue van with flames and an observation bubble.<br /><br />The cliché characters in ridiculous situations are what make this film such great fun. The wonderful comedic chemistry between Stephen Furst (Harold) and Andy Tennant (Melio) make up most of my favorite parts of the movie. And who didn't love the hopeless awkwardness of Flynch? Don't forget the airport antics of Leon's cronies, dressed up as Hari Krishnas: dancing, chanting and playing the tambourine--unbeatable! The clues are genius, the locations are classic, and the plot is timeless.<br /><br />A word to the wise, if you didn't watch this film when you were little, it probably won't win a place in your heart today. But nevertheless give it a chance, you may find that "It doesn't matter what you say, it doesn't matter what you do, you'v

In [29]:
cat_unique_val = {}

for category in categories:
    mask = text_df['text'] == category
    list_length = len(list(text_df[mask]['label'].unique()))
    cat_unique_val.update({category: list_length})

    cat_unique_val

In [30]:
cat_unique_val_sorted = {}
sorted_keys = sorted(cat_unique_val, key=cat_unique_val.get)

for w in list(reversed(sorted_keys)):
    cat_unique_val_sorted[w] = cat_unique_val[w]
    
# cat_unique_val_sorted
cat_unique_val_sorted

{'Why would this film be so good, but only gross an estimated $95,000,000 and have NO award nominations? John Travolta knows what he\'s doing. He knows he\'s Michael, a cigar smoking, womanizing, magical arch angel that came down to live with a dying lady and is now in a car with the staff of "The National Mirror" and their dog, Sparky, on the way to Chicago. It then turns into a road trip that\'s both horrible AND great. I don\'t even think the death scenes (3 to be exact) make this a tearjerker. The soundtrack is the best with "Heaven Is My Home", "Up Around The Bend", and "Chains Of Fools". I have very great expectations about this and I say that it should have had a little more respect in the 90\'s. Read my comment. Bye!': 1,
 'When I first watched Flatliners, I was amazed. It had all the necessary features of a good movie: the cast was superb, the plot was superb, and in the case of thrillers, there was genuine "thrills" throughout.<br /><br />Keifer Sutherland offered a marvelous

In [31]:
text_df = text_df.drop_duplicates()

In [32]:
text_df.shape

(39723, 2)

In [33]:
stop_words = set(stopwords.words('english'))

In [34]:
regular_punct = list(string.punctuation)

In [35]:
text_df['label']

0        0
1        0
2        0
3        0
4        1
        ..
39995    1
39996    1
39997    0
39998    1
39999    1
Name: label, Length: 39723, dtype: int64

In [36]:
def text_preprocessing(x):
    filtered_sentence = []
    word_tokens = word_tokenize(x)
    
    for w in word_tokens:
        if w not in chain(stop_words, regular_punct):
            # we make sure that all words are written in lowercase
            filtered_sentence.append(w.lower())
    
    # Converting a list of strings back to a string
    filtered_sentence = " ".join(filtered_sentence)
    return filtered_sentence

In [37]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
text_df['text'] = text_df['text'].apply(text_preprocessing)

In [39]:
text_df['text']

0        i grew b 1965 watching loving thunderbirds all...
1        when i put movie dvd player sat coke chips i e...
2        why people know particular time past like feel...
3        even though i great interest biblical movies i...
4        im die hard dads army fan nothing ever change ...
                               ...                        
39995    `` western union '' something forgotten classi...
39996    this movie incredible piece work it explores e...
39997    my wife i watched movie plan visit sicily stro...
39998    when i first watched flatliners i amazed it ne...
39999    why would film good gross estimated 95,000,000...
Name: text, Length: 39723, dtype: object

In [40]:
counts = feature_extraction.text.CountVectorizer()
X = counts.fit_transform(text_df['text'].values)
X.shape

(39723, 92893)

In [41]:
from sklearn.naive_bayes import MultinomialNB

In [42]:
y = text_df['text'].values

In [43]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#colab keeps crashing and showing RAM error

In [ ]:
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)


In [4]:
model_naive.score(X_train, y_train)

In [ ]:
model_naive.score(X_test, y_test)

In [ ]:
y_pred = model_naive.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
target_names = categories
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
mat = confusion_matrix(y_test, model_naive.predict(X_test))
plt.figure(figsize=(18,12))
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=categories,
            yticklabels=categories
           )
plt.xlabel('true label')
plt.ylabel('predicted label');